In [15]:
import cv2

# Initialize the video capture object to access the default webcam (0)
# If you have multiple cameras, you might need to use 1, 2, etc.
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

# Main loop to capture frames continuously
while True:
    # Read a new frame from the video stream
    ret, frame = cap.read()
    
    # 'ret' is a boolean that is True if the frame was read correctly
    if not ret:
        print("Error: Can't receive frame (stream end?). Exiting ...")
        break
        
    # Display the resulting frame in a window
    cv2.imshow('My Security Feed', frame)
    
    # Wait for a key press. If 'q' is pressed, break the loop.
    # The waitKey(1) means a 1ms delay, which makes the video play smoothly.
    if cv2.waitKey(1) == ord('q'):
        break

# When everything is done, release the capture and destroy all windows
cap.release()
cv2.destroyAllWindows()
print("Video stream stopped.")

Video stream stopped.


In [16]:
import numpy as np 

# --- Keep the initial video capture code from Phase 1 ---

# Create the background subtractor object
# MOG2 is a popular algorithm for this. Defaults are usually fine.
backSub = cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    # Apply the background subtractor to the frame to get the foreground mask
    fgMask = backSub.apply(frame)
    
    # Display the original frame
    cv2.imshow('My Security Feed', frame)
    
    # Display the foreground mask
    cv2.imshow('Foreground Mask', fgMask)
    
    if cv2.waitKey(1) == ord('q'):
        break
        


In [17]:

while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    fgMask = backSub.apply(frame)
    
    # Find contours in the mask
    # cv2.RETR_EXTERNAL finds only the outer contours.
    # cv2.CHAIN_APPROX_SIMPLE compresses horizontal, vertical, and diagonal segments.
    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Loop over each contour found in the mask
    for contour in contours:
        # Filter out small contours to avoid detecting noise
        # You can adjust this value based on your camera and environment
        if cv2.contourArea(contour) < 500:
            continue
        
        # If we are here, it means we have found a significant motion
        print("Motion detected!") # For now, we'll just print to the console
    
    cv2.imshow('My Security Feed', frame)
    cv2.imshow('Foreground Mask', fgMask)
    
    if cv2.waitKey(1) == ord('q'):
        break



In [18]:
import datetime

# --- Keep the setup code ---

while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    motion_detected = False # Flag to check if motion was detected in this frame
    fgMask = backSub.apply(frame)
    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        if cv2.contourArea(contour) < 500:
            continue
            
        # Get the bounding box for the contour
        (x, y, w, h) = cv2.boundingRect(contour)
        
        # Draw the rectangle on the original frame
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        motion_detected = True

    # Get the current timestamp and format it
    timestamp = datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p")
    
    # Add status text and timestamp to the frame
    if motion_detected:
        status_text = "Status: Motion Detected"
        cv2.putText(frame, status_text, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    else:
        status_text = "Status: No Motion"
        cv2.putText(frame, status_text, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
    cv2.putText(frame, timestamp, (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    cv2.imshow('My Security Feed', frame)
    # No need to show the mask anymore, but you can keep it for debugging
    # cv2.imshow('Foreground Mask', fgMask)
    
    if cv2.waitKey(1) == ord('q'):
        break


In [19]:
import cv2
import datetime
import time

# Initialize video capture and background subtractor
cap = cv2.VideoCapture(0)
backSub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)

# Cooldown variables
last_trigger_time = 0
cooldown_period = 5 # in seconds

print("Starting security feed... Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    motion_detected = False
    fgMask = backSub.apply(frame)
    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        if cv2.contourArea(contour) < 900: # Increased threshold
            continue
        
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        motion_detected = True

    current_time = time.time()
    
    if motion_detected:
        status_text = "Status: Motion Detected"
        cv2.putText(frame, status_text, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        # Check if cooldown has passed
        if (current_time - last_trigger_time) > cooldown_period:
            print(f"Motion detected! Saving snapshot...")
            
            # Create a filename with a unique timestamp
            timestamp_str = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            filename = f"motion_{timestamp_str}.jpg"
            
            # Save the frame
            cv2.imwrite(filename, frame)
            
            # Update the last trigger time
            last_trigger_time = current_time
            
    else:
        status_text = "Status: No Motion"
        cv2.putText(frame, status_text, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Add timestamp to the frame
    timestamp_display = datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p")
    cv2.putText(frame, timestamp_display, (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    cv2.imshow('My Security Feed', frame)
    
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("Application closed.")

Starting security feed... Press 'q' to quit.
Motion detected! Saving snapshot...
Motion detected! Saving snapshot...
Motion detected! Saving snapshot...
Motion detected! Saving snapshot...
Motion detected! Saving snapshot...
Motion detected! Saving snapshot...
Motion detected! Saving snapshot...
Motion detected! Saving snapshot...
Application closed.
